# Task 3.2: Production Feasibility Simulator

This notebook validates the production planning projections from Task 3.1 by simulating:
1. Daily inventory dynamics under proposed production rates
2. Achievement of 12-week 99% autonomy target
3. Stockout analysis and feasibility validation

## Simulation Logic:
- Start production at calculated prior_start_date
- Produce at steady daily rate (accounting for US holidays)
- Track inventory: Inventory(t+1) = Inventory(t) + Production(t) - Demand(t)
- Calculate rolling 84-day autonomy at each date
- Validate: Does it achieve 99% autonomy?

##  Data Sources:
- **Demand**: Task 2 simulator output (sim_batches/*.csv.gz)
- **Production rates**: Task 3.1 output (cluster_production_rates.csv)
- **Prior start days**: Task 3.1 output (prior_start_days_corrected.csv)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_style('whitegrid')

print("Production Feasibility Simulator - Task 3.2")
print("="*80)

## 1. Configuration and Constants

In [ ]:
# Constants
US_HOLIDAYS_PER_YEAR = 11
WORKDAYS_PER_YEAR = 365 - US_HOLIDAYS_PER_YEAR
AUTONOMY_DAYS = 84
AUTONOMY_THRESHOLD_PERCENTILE = 99

# Simplified US holiday model (assume distributed throughout year)
HOLIDAY_RATE = US_HOLIDAYS_PER_YEAR / 365

def is_workday(date):
    """Simplified: Use probability-based workday determination"""
    # In real implementation, would check against actual holiday calendar
    # For simulation, use deterministic pattern to match workdays_per_year
    day_of_year = date.dayofyear
    # Distribute 11 holidays roughly evenly: every ~33 days
    holiday_pattern = [1, 32, 64, 95, 126, 157, 188, 219, 250, 281, 312, 343]
    return day_of_year not in holiday_pattern

print(f"Configuration:")
print(f"  Workdays per year: {WORKDAYS_PER_YEAR}")
print(f"  Autonomy target: {AUTONOMY_DAYS} days")
print(f"  Autonomy threshold: P{AUTONOMY_THRESHOLD_PERCENTILE}\n")

## 2. Load Production Planning Inputs

In [ ]:
# Load production rates from Task 3.1
production_rates_df = pd.read_csv('cluster_production_rates.csv')
print("Production rates loaded:")
print(production_rates_df[['year', 'cluster', 'daily_production_99robust']].head(10))

# Load prior start days from Task 3.1
prior_start_df = pd.read_csv('prior_start_days_corrected.csv')
prior_start_df['production_start_date'] = pd.to_datetime(prior_start_df['production_start_date'])
print("\nPrior start days loaded:")
print(prior_start_df[['year', 'cluster', 'prior_start_days', 'production_start_date']].head(10))

## 3. Load Demand Simulations

In [ ]:
# Load all simulation batches
print("\nLoading demand simulations...")
sim_batches_path = Path('sim_batches')
batch_files = sorted(sim_batches_path.glob('batch_*.csv.gz'))

# Load first 10 simulations for faster testing (can increase for full validation)
NUM_SIMS_TO_TEST = 20  # Use 20 simulations for testing

sim_data = []
for batch_file in batch_files[:NUM_SIMS_TO_TEST]:
    df = pd.read_csv(batch_file, compression='gzip')
    sim_data.append(df)
    print(f"  Loaded {batch_file.name}")

sim_data = pd.concat(sim_data, ignore_index=True)
sim_data['date'] = pd.to_datetime(sim_data['date'])
sim_data['year'] = sim_data['date'].dt.year

# Add cluster assignment
def get_cluster(model):
    category = model[0]
    if category in ['F', 'K', 'L']:
        return 'Cluster_1'
    elif category in ['S', 'W', 'X']:
        return 'Cluster_2'
    return 'Unknown'

sim_data['cluster'] = sim_data['model'].apply(get_cluster)

# Aggregate by cluster
daily_cluster_sales = sim_data.groupby(['sim', 'date', 'cluster'])['sales_units'].sum().reset_index()

print(f"\nLoaded {sim_data['sim'].nunique()} simulations")
print(f"Date range: {sim_data['date'].min()} to {sim_data['date'].max()}")
print(f"Daily cluster sales: {len(daily_cluster_sales):,} rows")

## 4. Production Simulator Function

In [ ]:
def simulate_production_and_inventory(sim_id, cluster, demand_df, production_plan, prior_start_info):
    """
    Simulate daily production and inventory for one simulation and cluster.
    
    Args:
        sim_id: Simulation ID
        cluster: Cluster name
        demand_df: Daily demand data for this sim/cluster
        production_plan: DataFrame with production rates by year
        prior_start_info: DataFrame with prior start days by year
    
    Returns:
        DataFrame with daily inventory, production, demand, and autonomy metrics
    """
    results = []
    
    # Get demand data for this simulation and cluster
    sim_demand = demand_df[
        (demand_df['sim'] == sim_id) & 
        (demand_df['cluster'] == cluster)
    ].sort_values('date').copy()
    
    if len(sim_demand) == 0:
        return pd.DataFrame()
    
    # Get all years in this simulation
    years = sorted(sim_demand['year'].unique())
    
    # Initialize inventory
    inventory = 0
    
    # Create extended date range including prior production periods
    first_year = years[0]
    first_year_prior = prior_start_info[
        (prior_start_info['year'] == first_year) & 
        (prior_start_info['cluster'] == cluster)
    ]
    
    if len(first_year_prior) > 0:
        start_date = first_year_prior['production_start_date'].values[0]
        start_date = pd.Timestamp(start_date)
    else:
        # Fallback: start 90 days before first year
        start_date = pd.Timestamp(f'{first_year}-01-01') - pd.Timedelta(days=90)
    
    end_date = sim_demand['date'].max()
    
    # Create complete date range
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Simulate day by day
    for current_date in date_range:
        current_year = current_date.year
        
        # Determine if we should produce today (workday check)
        produce_today = is_workday(current_date)
        
        # Get production rate for current year
        # If we're in prior period, use the year we're producing for
        production_year = current_year
        if current_date < pd.Timestamp(f'{current_year}-01-01'):
            production_year = current_year  # Producing for upcoming year
        
        prod_rate_data = production_plan[
            (production_plan['year'] == production_year) & 
            (production_plan['cluster'] == cluster)
        ]
        
        if len(prod_rate_data) > 0 and produce_today:
            daily_production = prod_rate_data['daily_production_99robust'].values[0]
        else:
            daily_production = 0
        
        # Get demand for today (0 if before year starts or no data)
        demand_today = 0
        if current_date >= pd.Timestamp(f'{first_year}-01-01'):
            demand_row = sim_demand[sim_demand['date'] == current_date]
            if len(demand_row) > 0:
                demand_today = demand_row['sales_units'].values[0]
        
        # Update inventory
        inventory = inventory + daily_production - demand_today
        
        # Calculate 84-day forward demand (for autonomy check)
        future_dates = pd.date_range(start=current_date, periods=AUTONOMY_DAYS, freq='D')
        future_demand = sim_demand[
            sim_demand['date'].isin(future_dates)
        ]['sales_units'].sum()
        
        # Calculate autonomy (days of demand covered)
        if future_demand > 0:
            autonomy_ratio = inventory / future_demand
        else:
            autonomy_ratio = np.inf if inventory > 0 else 0
        
        has_autonomy = inventory >= future_demand
        
        results.append({
            'sim': sim_id,
            'cluster': cluster,
            'date': current_date,
            'year': production_year,
            'production': daily_production,
            'demand': demand_today,
            'inventory': inventory,
            'future_84day_demand': future_demand,
            'autonomy_ratio': min(autonomy_ratio, 2.0),  # Cap at 2x for plotting
            'has_84day_autonomy': has_autonomy,
            'is_workday': produce_today
        })
    
    return pd.DataFrame(results)

print("✓ Simulator function defined")

## 5. Run Simulation

In [ ]:
print("\n" + "="*80)
print("RUNNING PRODUCTION FEASIBILITY SIMULATION")
print("="*80)

simulation_results = []

sim_ids = sorted(daily_cluster_sales['sim'].unique())
clusters = ['Cluster_1', 'Cluster_2']

total_sims = len(sim_ids) * len(clusters)
current = 0

for sim_id in sim_ids:
    for cluster in clusters:
        current += 1
        print(f"\rSimulating {current}/{total_sims}: Sim {sim_id}, {cluster}...", end='')
        
        sim_result = simulate_production_and_inventory(
            sim_id=sim_id,
            cluster=cluster,
            demand_df=daily_cluster_sales,
            production_plan=production_rates_df,
            prior_start_info=prior_start_df
        )
        
        if len(sim_result) > 0:
            simulation_results.append(sim_result)

print("\n\nCombining results...")
all_results = pd.concat(simulation_results, ignore_index=True)

print(f"\n✓ Simulation complete: {len(all_results):,} daily records generated")
print(f"  Simulations: {all_results['sim'].nunique()}")
print(f"  Clusters: {all_results['cluster'].nunique()}")
print(f"  Date range: {all_results['date'].min()} to {all_results['date'].max()}")

## 6. Analyze Results

In [ ]:
print("\n" + "="*80)
print("FEASIBILITY ANALYSIS")
print("="*80)

# Filter to operational dates only (after year start)
operational_results = all_results[all_results['date'] >= pd.Timestamp('2027-01-01')].copy()

# Calculate autonomy achievement rate
autonomy_stats = operational_results.groupby(['year', 'cluster']).agg({
    'has_84day_autonomy': ['mean', 'sum', 'count'],
    'inventory': ['mean', 'min', 'max'],
    'autonomy_ratio': ['mean', lambda x: np.percentile(x[np.isfinite(x)], 5)]
}).reset_index()

autonomy_stats.columns = ['year', 'cluster', 'autonomy_achievement_rate', 
                          'days_with_autonomy', 'total_days',
                          'avg_inventory', 'min_inventory', 'max_inventory',
                          'avg_autonomy_ratio', 'p05_autonomy_ratio']

print("\nAutonomy Achievement by Year and Cluster:")
print("="*80)
print(autonomy_stats.to_string(index=False))

# Save results
autonomy_stats.to_csv('autonomy_achievement_stats.csv', index=False)
print("\n✓ Saved autonomy_achievement_stats.csv")

In [ ]:
# Check for stockouts
print("\n" + "="*80)
print("STOCKOUT ANALYSIS")
print("="*80)

stockouts = operational_results[operational_results['inventory'] < 0]

if len(stockouts) > 0:
    print(f"\n⚠️  WARNING: {len(stockouts)} stockout incidents detected!")
    print("\nStockout summary by year and cluster:")
    stockout_summary = stockouts.groupby(['year', 'cluster']).size().reset_index(name='stockout_days')
    print(stockout_summary)
else:
    print("\n✅ NO STOCKOUTS DETECTED - Production plan is feasible!")

# Check days without autonomy
no_autonomy = operational_results[~operational_results['has_84day_autonomy']]
print(f"\nDays without 84-day autonomy: {len(no_autonomy):,} / {len(operational_results):,}")
print(f"Autonomy failure rate: {len(no_autonomy)/len(operational_results)*100:.2f}%")

if len(no_autonomy) > 0:
    print("\nAutonomy failures by year and cluster:")
    failure_summary = no_autonomy.groupby(['year', 'cluster']).size().reset_index(name='failure_days')
    print(failure_summary)

## 7. Visualizations

In [ ]:
# Plot 1: Inventory Levels Over Time (2027 example)
print("\nGenerating visualizations...")

year_to_plot = 2027
year_data = operational_results[operational_results['year'] == year_to_plot]

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

for idx, cluster in enumerate(['Cluster_1', 'Cluster_2']):
    cluster_data = year_data[year_data['cluster'] == cluster]
    
    # Calculate daily statistics across simulations
    daily_stats = cluster_data.groupby('date').agg({
        'inventory': ['mean', lambda x: np.percentile(x, 5), lambda x: np.percentile(x, 95)]
    }).reset_index()
    daily_stats.columns = ['date', 'inventory_mean', 'inventory_p05', 'inventory_p95']
    
    axes[idx].plot(daily_stats['date'], daily_stats['inventory_mean'], 
                   linewidth=2, label='Mean inventory', color='blue')
    axes[idx].fill_between(daily_stats['date'], 
                           daily_stats['inventory_p05'], 
                           daily_stats['inventory_p95'],
                           alpha=0.3, label='P05-P95 range')
    axes[idx].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Zero inventory')
    
    axes[idx].set_xlabel('Date', fontsize=12)
    axes[idx].set_ylabel('Inventory (units)', fontsize=12)
    axes[idx].set_title(f'{cluster} Inventory Levels ({year_to_plot})', 
                       fontsize=14, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('inventory_levels_simulation.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved inventory_levels_simulation.png")

In [ ]:
# Plot 2: Autonomy Achievement Rate by Year
fig, ax = plt.subplots(figsize=(12, 6))

years = sorted(autonomy_stats['year'].unique())
x = np.arange(len(years))
width = 0.35

cluster1_data = autonomy_stats[autonomy_stats['cluster'] == 'Cluster_1'].sort_values('year')
cluster2_data = autonomy_stats[autonomy_stats['cluster'] == 'Cluster_2'].sort_values('year')

ax.bar(x - width/2, cluster1_data['autonomy_achievement_rate'] * 100, width, 
       label='Cluster 1 (F, K, L)', alpha=0.8, color='steelblue')
ax.bar(x + width/2, cluster2_data['autonomy_achievement_rate'] * 100, width, 
       label='Cluster 2 (S, W, X)', alpha=0.8, color='coral')

ax.axhline(y=99, color='green', linestyle='--', linewidth=2, label='99% Target')

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Autonomy Achievement Rate (%)', fontsize=12)
ax.set_title('12-Week Autonomy Achievement Rate by Year and Cluster', 
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(years)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 105])

plt.tight_layout()
plt.savefig('autonomy_achievement_rate.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved autonomy_achievement_rate.png")

In [ ]:
# Plot 3: Autonomy Ratio Distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for idx, cluster in enumerate(['Cluster_1', 'Cluster_2']):
    cluster_data = operational_results[
        (operational_results['cluster'] == cluster) &
        (operational_results['autonomy_ratio'] < 2)
    ]
    
    axes[idx].hist(cluster_data['autonomy_ratio'], bins=50, alpha=0.7, 
                  color='steelblue' if idx == 0 else 'coral', edgecolor='black')
    axes[idx].axvline(x=1.0, color='green', linestyle='--', linewidth=2, 
                     label='Full autonomy (ratio=1.0)')
    
    axes[idx].set_xlabel('Autonomy Ratio (Inventory / 84-day Demand)', fontsize=12)
    axes[idx].set_ylabel('Frequency', fontsize=12)
    axes[idx].set_title(f'{cluster} Autonomy Ratio Distribution', 
                       fontsize=14, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('autonomy_ratio_distribution.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved autonomy_ratio_distribution.png")

## 8. Final Validation Report

In [ ]:
print("\n" + "="*120)
print("TASK 3.2 VALIDATION REPORT: PRODUCTION PLAN FEASIBILITY")
print("="*120)

# Overall statistics
overall_autonomy_rate = operational_results['has_84day_autonomy'].mean()
overall_stockout_rate = (operational_results['inventory'] < 0).mean()

print("\n📊 OVERALL PERFORMANCE:\n")
print(f"  Overall autonomy achievement rate: {overall_autonomy_rate*100:.2f}%")
print(f"  Stockout rate: {overall_stockout_rate*100:.4f}%")
print(f"  Simulations tested: {all_results['sim'].nunique()}")
print(f"  Total days simulated: {len(operational_results):,}")

# Validation result
print("\n" + "="*120)
if overall_autonomy_rate >= 0.99 and overall_stockout_rate == 0:
    print("\n✅ VALIDATION PASSED!")
    print("\n   The production plan achieves:")
    print("   ✓ 99%+ autonomy achievement rate")
    print("   ✓ Zero stockouts")
    print("   ✓ Feasible inventory levels")
    print("\n   Production parameters are VALIDATED for implementation.")
elif overall_autonomy_rate >= 0.95:
    print("\n⚠️  VALIDATION PARTIALLY PASSED")
    print(f"\n   Autonomy achievement: {overall_autonomy_rate*100:.2f}% (target: 99%)")
    print("   Recommendation: Minor adjustments may be needed")
    print("   Consider:")
    print("   - Increasing prior start days by 5-10%")
    print("   - Adding small safety buffer to production rate")
else:
    print("\n❌ VALIDATION FAILED")
    print(f"\n   Autonomy achievement: {overall_autonomy_rate*100:.2f}% (target: 99%)")
    print("   Recommendation: Significant adjustments needed")
    print("   Action items:")
    print("   1. Increase prior start days")
    print("   2. Review critical date identification")
    print("   3. Consider higher production buffer")

print("\n" + "="*120)
print("\n📁 OUTPUT FILES GENERATED:")
print("   - autonomy_achievement_stats.csv")
print("   - inventory_levels_simulation.png")
print("   - autonomy_achievement_rate.png")
print("   - autonomy_ratio_distribution.png")
print("\n" + "="*120)

# Detailed recommendations by cluster
print("\n📋 DETAILED RECOMMENDATIONS BY CLUSTER:\n")
for cluster in ['Cluster_1', 'Cluster_2']:
    cluster_stats = autonomy_stats[autonomy_stats['cluster'] == cluster]
    avg_achievement = cluster_stats['autonomy_achievement_rate'].mean()
    min_inventory = cluster_stats['min_inventory'].min()
    
    print(f"  {cluster}:")
    print(f"    Average autonomy achievement: {avg_achievement*100:.2f}%")
    print(f"    Minimum inventory observed: {min_inventory:,.2f} units")
    
    if avg_achievement >= 0.99:
        print(f"    ✅ Meets target - Plan is validated")
    elif avg_achievement >= 0.95:
        gap = (0.99 - avg_achievement) * 100
        print(f"    ⚠️  Close to target (gap: {gap:.2f}%) - Minor adjustment recommended")
    else:
        gap = (0.99 - avg_achievement) * 100
        print(f"    ❌ Below target (gap: {gap:.2f}%) - Requires attention")
    print()

print("="*120)